In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Directive pour afficher les graphiques dans Jupyter
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn import model_selection

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split

J'importe premièrement les datasets puis je les join.

In [ ]:
df0 = pd.read_csv('../input/crimes-in-chicago/Chicago_Crimes_2001_to_2004.csv', error_bad_lines=False)
df1 = pd.read_csv('../input/crimes-in-chicago/Chicago_Crimes_2005_to_2007.csv', error_bad_lines=False)
df2 = pd.read_csv('../input/crimes-in-chicago/Chicago_Crimes_2008_to_2011.csv', error_bad_lines=False)
df3 = pd.read_csv('../input/crimes-in-chicago/Chicago_Crimes_2012_to_2017.csv', error_bad_lines=False)

frames = [df0, df1, df2, df3]

df = pd.concat(frames)

In [ ]:
df.tail()

Je convertie ensuite les dates au format `string` en `datetime`.

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

Je tronque ensuite l'ensemble des dates du dataset pour ne garder que les jours, mois, années.

In [ ]:
import datetime
df['Date'] = df['Date'].apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day))

In [ ]:
df.head()

Je trie ensuite l'ensemble des datas dans l'ordre croissante des dates.

In [ ]:
df = df.sort_values(by = 'Date')

In [ ]:
df.head()

Puis je garde uniquement ce qui m'intérèsse, ici les dates et les types de crimes commis.

In [ ]:
df_dates = df[['Date', 'Primary Type']]
df_dates.head()

Je groupe ensuite les données selon leur date et je compte le nombre de valeurs

In [ ]:
df_dates_count = df_dates.groupby(['Date']).count().reset_index()
df_dates_count.head()

Si on affiche le dataset en l'état actuel, on remarque une certaine périodicité au fil de l'année.

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(df_dates_count['Date'] , df_dates_count['Primary Type'])

Je transforme ensuite mon dataset pour qu'il puisse être traiter par Prophet

In [ ]:
from fbprophet import Prophet
df_prophet = df_dates_count.copy()
df_prophet.columns = ['ds','y']
df_prophet['ds'] = pd.to_datetime(df_prophet['ds'])
print (df_prophet.dtypes)

Je sépare mon dataset en deux, un d'entraînement et un de test

In [ ]:
train = df_prophet[df_prophet['ds'] < '2014-07-01'].copy()
test = df_prophet[df_prophet['ds'] >= '2014-07-01'].copy()

En bleu l'entrainement, en gris le test

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(train['ds'] , train['y'], color = 'blue')
plt.plot(test['ds'] , test['y'], color = 'grey')
plt.show()

Enfin je lance la prédiction 

In [ ]:
prophet = Prophet()
prophet.fit(train)
prediction = prophet.predict(test)

EN orange on peut voir la prédiction de Prophet. C'est plutôt correct

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(train['ds'] , train['y'], color = 'blue')
plt.plot(test['ds'] , test['y'], color = 'grey', alpha = 0.5)
plt.plot(prediction['ds'] , prediction['yhat'], color = 'orange', alpha = 0.5)
plt.show()

Par contre les bornes minimales et maximales sont étranges...

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(train['ds'] , train['y'], color = 'blue')
plt.plot(test['ds'] , test['y'], color = 'grey', alpha = 0.5)
plt.plot(prediction['ds'] , prediction['yhat_upper'], color = 'red', alpha = 0.5)
plt.plot(prediction['ds'] , prediction['yhat_lower'], color = 'green', alpha = 0.5)
plt.show()